In [1]:
import numpy as np
import os


def step_source(t):
    # return 0.0
    # return np.exp(-0.5 * ((t - 0.005) / 0.0001) ** 2) if t < 0.01 else 0.0
    #return np.sin((np.pi / 0.01) * t) ** 4 if t < 0.01 else 0.0
    return 100.0 * float(t < 0.01)

In [2]:
import sympy as sym
sym.init_printing()

def get_elastic_matrices(Cp, Cs, Rho):

    var_rho   = Rho
    var_lmbda = (Cp ** 2. - 2. * Cs ** 2) * var_rho
    var_mu    = Cs ** 2. * var_rho

    lmbda, mu, rho =  sym.symbols('\lambda \mu \rho')

    elastic_x = sym.Matrix([
        [0., 0., 1/rho, 0., 0.],
        [0., 0., 0., 0., 1/rho],
        [lmbda + 2 * mu, 0., 0., 0., 0.],
        [lmbda, 0., 0., 0., 0.],
        [0., mu, 0., 0., 0.],
    ])

    elastic_y = sym.Matrix([
        [0., 0., 0., 0., 1/rho],
        [0., 0., 0., 1/rho, 0.],
        [0., lmbda, 0., 0., 0.],
        [0., lmbda + 2 * mu, 0., 0., 0.],
        [mu, 0., 0., 0., 0.],
    ])


    P1, D1 = elastic_x.diagonalize()
    P2, D2 = elastic_y.diagonalize()
    
    print(D1)

    Ux, Sx, Ux1  = np.zeros((*Cp.shape, 5, 5)), np.zeros((*Cp.shape, 5, 5)), np.zeros((*Cp.shape, 5, 5))
    Uy, Sy, Uy1  = np.zeros((*Cp.shape, 5, 5)), np.zeros((*Cp.shape, 5, 5)), np.zeros((*Cp.shape, 5, 5))
    
    
    for i, (p1, d1, p11) in enumerate(zip(P1, D1, P1.inv())):
        
        fp1  = sym.lambdify([lmbda, mu, rho], p1, 'numpy')
        fd1  = sym.lambdify([lmbda, mu, rho], d1, 'numpy')
        fp11 = sym.lambdify([lmbda, mu, rho], p11, 'numpy')
      
        # TODO: fill in the gaps
        Ux[:, :, i//5,i%5]  = fp1(var_lmbda, var_mu, var_rho)
        Sx[:, :, i//5,i%5]  = fd1(var_lmbda, var_mu, var_rho)
        Ux1[:, :, i//5,i%5] = fp11(var_lmbda, var_mu, var_rho)

    for i, (p2, d2, p21) in enumerate(zip(P2, D2, P2.inv())):
        
        fp2  = sym.lambdify([lmbda, mu, rho], p2, 'numpy')
        fd2  = sym.lambdify([lmbda, mu, rho], d2, 'numpy')
        fp21 = sym.lambdify([lmbda, mu, rho], p21, 'numpy')
      
        # TODO: fill in the gaps
        Uy[:, :, i//5,i%5]  = fp2(var_lmbda, var_mu, var_rho)
        Sy[:, :, i//5,i%5]  = fd2(var_lmbda, var_mu, var_rho)
        Uy1[:, :, i//5,i%5] = fp21(var_lmbda, var_mu, var_rho)

    return (
        Ux, np.diagonal(Sx, axis1=-2, axis2=-1), 
        Ux1, Uy, np.diagonal(Sy, axis1=-2, axis2=-1), Uy1
    )
            

In [3]:
from pyevtk.hl import gridToVTK

class elastics_solver():
    
    def __init__(
        self, 
        x_size, y_size, 
        cp, cs, rho, 
        target_time, 
        recording_time_step,
        source_width, 
        source_function = step_source, 
        source_center_in_percents = 50,
        dump_vtk = False, 
        dump_dir = "data", 
        verbose = False
    ):

        assert (cp.shape == rho.shape)

        self.dump_vtk = dump_vtk
        self.dump_dir = dump_dir
        self.verbose = verbose
    
        self.x_size = x_size
        self.y_size = y_size
        
        
        self.num_points_x, self.num_points_y = cp.shape
        self.hx = self.x_size / (self.num_points_x - 1)
        self.hy = self.y_size / (self.num_points_y - 1)

        # required for numerical stability
        assert (abs(self.hx - self.hy) < 0.01 * (self.hx + self.hy))

        self.cp = cp
        self.cs = cs
        self.rho = rho
        
        self.lmbda = rho * (cp ** 2 - 2*cs ** 2)
        self.mu    = rho * cs ** 2
        
        # self.K = np.square(self.cp) * self.rho

        self.T = 0

        max_cp = np.max(self.cp)
        numerical_method_recommended_tau = 0.45 * min(self.hx / max_cp, self.hy / max_cp)

        if self.verbose:
            print("Numerical time step recommendation:", numerical_method_recommended_tau)

        self.number_of_records = int(target_time / recording_time_step)
        self.steps_per_record = max(int(recording_time_step / numerical_method_recommended_tau), 1)
        self.tau = recording_time_step / self.steps_per_record

        if self.verbose:
            print("Doing %d data records, %d steps per record, total %d steps, time step is %f, final time %f" %
                  (self.number_of_records, self.steps_per_record,
                   self.number_of_records * self.steps_per_record, self.tau, target_time))

        self.x = np.linspace(0.0, self.x_size, self.num_points_x, endpoint=True)
        self.y = np.linspace(0.0, self.y_size, self.num_points_y, endpoint=True)
        self.z = np.array([0.0])

        if self.dump_vtk:
            gridToVTK(os.path.join(self.dump_dir, "params"), self.x, self.y, self.z,
                      pointData = {
                          "Cp" : self.cp.T.ravel(), 
                          "Cs" : self.cs.T.ravel(),
                          "rho" : self.rho.T.ravel()
                      })

        source_half_width_in_points = int(source_width / (2 * self.hx))
        source_center_idx = int(self.num_points_x * source_center_in_percents / 100)
        self.source_start_point = source_center_idx - source_half_width_in_points
        self.source_end_point = source_center_idx + source_half_width_in_points

        self.source = source_function
        if self.verbose:
            print("Grid shape", cp.shape)
            print("The source is located from %d to %d" % (self.source_start_point, self.source_end_point))
            
    
    def do_split_step(self, u_prev, U, U1, C, direction = -1):
        
        # TODO: implement explicit time-stepping using NumPy
        
        steps = C * self.tau 
        
        C1 = steps * (- self.hx + steps) / (2 * self.hx * self.hx)
        C2 = (self.hx + steps) * (- self.hx + steps) / (- self.hx * self.hx)
        C3 = (self.hx + steps) * steps / (2 * self.hx * self.hx)
        
        if direction == 0:
            u_lefts = np.pad(u_prev, ((1, 0), (0, 0), (0, 0)), mode='constant', constant_values=0)[:-1, :, :]
            u_rights = np.pad(u_prev, ((0, 1), (0, 0), (0, 0)), mode='constant', constant_values=0)[1:, :, :]
        elif direction == 1:
            u_lefts = np.pad(u_prev, ((0, 0), (1, 0), (0, 0)), mode='constant', constant_values=0)[:, :-1, :]
            u_rights = np.pad(u_prev, ((0, 0), (0, 1), (0, 0)), mode='constant', constant_values=0)[:, 1:, :]
        else:
            raise RuntimeError("Incorrect axis for 2D method")
            
        riemans_here  = np.einsum('qikl,qil->qik', U1, u_prev)
        riemans_left  = np.einsum('qikl,qil->qik', U1, u_lefts)
        riemans_right = np.einsum('qikl,qil->qik', U1, u_rights)
        
        limiter_min = np.min([riemans_here, riemans_left, riemans_right], axis=0)
        limiter_max = np.max([riemans_here, riemans_left, riemans_right], axis=0)
        
        riemans_next = C1*riemans_left + C2*riemans_here + C3*riemans_right
        
        riemans_next = np.max([riemans_next, limiter_min], axis=0)
        riemans_next = np.min([riemans_next, limiter_max], axis=0)
        
        u_next = np.einsum('qikl,qil->qik', U, riemans_next)
        
        return u_next, riemans_next
    
    def forward(self):
        
        # TODO: implement forward wave propagation using space-split scheme
        # We are referring to https://keldysh.ru/council/3/D00202403/kazakov_ao_diss.pdf
        # as our basis
                
        Ux, Cx, Ux1, Uy, Cy, Uy1 = get_elastic_matrices(self.cp, self.cs, self.rho)
        
        u_prev = np.zeros((self.num_points_x, self.num_points_y, 5))
        

        # Time stepping using space-split scheme.
        for i in range(self.number_of_records):
            if self.verbose:
                print("Stepping for the time record", i)

            for j in range(self.steps_per_record):
                if self.verbose:
                    print("Step", j)

                # Two steps of 2D space-split scheme
                u_next, inv_next = self.do_split_step(u_prev, Uy, Uy1, Cy, direction=1)
                u_next, inv_next = self.do_split_step(u_next, Ux, Ux1, Cx, direction=0)

                # emitting and reflecting top border
                form = self.source(self.T)
                
                p0 = np.zeros(self.num_points_x)
                p0[self.source_start_point: self.source_end_point] = form
                
                u_next[:,-1,0] += u_next[:,-1, 4]/ np.sqrt(self.mu[:,-1] * self.rho[:,-1])
                u_next[:,-1,3] = p0
                u_next[:,-1,4] = 0.
                

                if self.dump_vtk:
                    gridToVTK(
                        os.path.join(self.dump_dir, "u" + str(i + 1)), self.x, self.y, self.z,
                        pointData={
                            "vx": u_next[:,:,0].T.ravel(),
                            "vy": u_next[:,:,1].T.ravel(),
                            "sxx": u_next[:,:,2].T.ravel(),
                            "syy": u_next[:,:,3].T.ravel(),
                            "sxy": u_next[:,:,4].T.ravel(),
                            "vel_magnitude": np.sqrt(u_next[:,:,0].T.ravel() ** 2 + u_next[:,:,1].T.ravel() ** 2) 
                        }
                    )

                u_prev = np.copy(u_next)

                self.T += self.tau
        
        return True

In [6]:
Cp = np.concatenate([
    np.ones((101, 51)) * 3000.,
    np.ones((101, 50)) * 4000.
], axis=1)

Cs = Cp / 1000000.
rho = np.ones_like(Cp) * 1000.

x_size = 1000.
y_size = 1000.

solver = elastics_solver(
    x_size, y_size, 
    Cp, Cs, rho, 
    2.0, 
    0.01,
    100.,
    dump_vtk = True, 
    dump_dir = "data", 
    verbose = True
)

Numerical time step recommendation: 0.0011250000000000001
Doing 200 data records, 8 steps per record, total 1600 steps, time step is 0.001250, final time 2.000000
Grid shape (101, 101)
The source is located from 45 to 55


In [7]:
solver.forward()

Matrix([[0, 0, 0, 0, 0], [0, -sqrt(\mu/ho), 0, 0, 0], [0, 0, sqrt(\mu/ho), 0, 0], [0, 0, 0, -sqrt((\lambda + 2*\mu)/ho), 0], [0, 0, 0, 0, sqrt((\lambda + 2*\mu)/ho)]])
Stepping for the time record 0
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 1
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 2
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 3
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 4
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 5
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 6
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 7
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 8
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 9
Step 0
Step 1
Step

Step 6
Step 7
Stepping for the time record 93
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 94
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 95
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 96
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 97
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 98
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 99
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 100
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 101
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 102
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 103
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for t

Step 4
Step 5
Step 6
Step 7
Stepping for the time record 187
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 188
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 189
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 190
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 191
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 192
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 193
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 194
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 195
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 196
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Stepping for the time record 197
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6


True